In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1')

In [9]:
pwd

'/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1'

In [10]:
import pandas as pd
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [11]:
# !wget -O data t.ly/Wh06

In [ ]:
!unzip data

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/image1938.jpg  
  inflating: dataset/train/image7383.jpg  
  inflating: dataset/train/image5498.jpg  
  inflating: dataset/train/image6955.jpg  
  inflating: dataset/train/image4840.jpg  
  inflating: dataset/train/image2695.jpg  
  inflating: dataset/train/image524.jpg  
  inflating: dataset/train/image1074.jpg  
  inflating: dataset/train/image3524.jpg  
  inflating: dataset/train/image3570.jpg  
  inflating: dataset/train/image8561.jpg  
  inflating: dataset/train/image5242.jpg  
  inflating: dataset/train/image567.jpg  
  inflating: dataset/train/image8431.jpg  
  inflating: dataset/train/image529.jpg  
  inflating: dataset/train/image5156.jpg  
  inflating: dataset/train/image6005.jpg  
  inflating: dataset/train/image6678.jpg  
  inflating: dataset/train/image9808.jpg  
  inflating: dataset/train/image1214.jpg  
  inflating: dataset/train/image6836.jpg  
  inflating: dataset/train/image949.jpg  
  infla

In [ ]:
df = pd.read_csv('./dataset/train.csv')
df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [ ]:
df['Class'].value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [ ]:
mapp = {}
rev_mapp = {}
curr = 0
for label in df['Class'].unique():
    mapp[label] = curr
    rev_mapp[curr] = label
    curr += 1

In [ ]:
temp_data = {}
for index, row in df.iterrows():
    temp_data[row['Image']] = row['Class']

In [ ]:
# temp_data

In [ ]:
os.listdir()

['weights.h5',
 'data',
 'Transfer learning.ipynb',
 'BasicCNN.ipynb',
 'Resnet50.ipynb',
 'dataset',
 'out.csv',
 'EfficientNet.ipynb']

## Training set

In [ ]:
ih = 200
iw = 180

In [ ]:
path = './dataset/train'
X_train = []
y_train = []

In [ ]:
from tqdm import tqdm
for file in tqdm(os.listdir(path)):
    img = image.load_img(os.path.join(path, file), target_size = (ih,iw))
    x = image.img_to_array(img)
    X_train.append(x)
    y_train.append(mapp[temp_data[file]])

KeyboardInterrupt: ignored

In [ ]:
X_train = np.stack(X_train)
y_train = np.stack(y_train)

In [ ]:
print(X_train.shape, y_train.shape)

(6469, 100, 70, 3) (6469,)


In [ ]:
from sklearn.model_selection import train_test_split
XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, test_size = 0.05)

In [ ]:
print(XX_train.shape, XX_test.shape)

(6145, 100, 70, 3) (324, 100, 70, 3)


In [ ]:
path = './dataset/test'
test_X = []
file_name = []

In [ ]:
for file in tqdm(os.listdir(path)):
    file_name.append(file)
    img = image.load_img(os.path.join(path, file), target_size = (ih, iw))
    x = image.img_to_array(img)
    test_X.append(x)

100%|██████████| 3489/3489 [00:04<00:00, 845.61it/s]


In [ ]:
test_X = np.stack(test_X)

In [ ]:
test_X.shape

(3489, 100, 70, 3)

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('acc') >= .9999):
            print("Cancelling Training")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
!pip install -U efficientnet

Requirement already up-to-date: efficientnet in /usr/local/lib/python3.6/dist-packages (1.1.1)


In [ ]:
import efficientnet.keras as efn

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

In [ ]:
base_model = Sequential()
base_model.add(efn.EfficientNetB0(include_top = False, weights='imagenet', pooling='max'))
# x = Flatten()(x)
x = Dense(128,  activation = 'relu')(x)
x = Dense(512, activation = 'relu')(x)
base_model.add(Dense(6, activation='softmax'))
base_model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
base_model.fit(XX_train, yy_train, epochs = 10, batch_size = 16, validation_data=(XX_test, yy_test), callbacks = [callbacks])
# base_model.fit(X_train, y_train, epochs = 10, batch_size = 16, callbacks = [callbacks])

Epoch 1/10
385/385 [==============================] - 15s 40ms/step - loss: 0.1110 - acc: 0.9634 - val_loss: 0.7958 - val_acc: 0.7901
Epoch 2/10
385/385 [==============================] - 15s 39ms/step - loss: 0.1280 - acc: 0.9590 - val_loss: 0.6286 - val_acc: 0.8395
Epoch 3/10
385/385 [==============================] - 15s 39ms/step - loss: 0.1327 - acc: 0.9561 - val_loss: 0.7920 - val_acc: 0.7932
Epoch 4/10
385/385 [==============================] - 15s 39ms/step - loss: 0.1472 - acc: 0.9523 - val_loss: 0.6616 - val_acc: 0.8395
Epoch 5/10
385/385 [==============================] - 15s 39ms/step - loss: 0.0644 - acc: 0.9808 - val_loss: 0.7646 - val_acc: 0.7963
Epoch 6/10
385/385 [==============================] - 15s 39ms/step - loss: 0.1156 - acc: 0.9621 - val_loss: 0.9708 - val_acc: 0.7994
Epoch 7/10
385/385 [==============================] - 15s 40ms/step - loss: 0.1441 - acc: 0.9536 - val_loss: 0.6566 - val_acc: 0.8179
Epoch 8/10
385/385 [==============================] - 15s 39ms

## Prediction

In [ ]:
y_hat = base_model.predict(test_X)

In [ ]:
test_y = []

In [ ]:
for lis in y_hat:
    index = np.argmax(lis)
    test_y.append(rev_mapp[index])

In [ ]:
print(test_y[:5])

['Jacket', 'Miscellaneous', 'Jacket', 'Christmas_Tree', 'Miscellaneous']


In [ ]:
res = {}
for i in range(test_X.shape[0]):
    res[file_name[i]] = test_y[i]

In [ ]:
result = pd.DataFrame(list(res.items()), columns = ['Image', 'Class'])

In [ ]:
result.head()

,Image,Class
0,image8132.jpg,Jacket
1,image1490.jpg,Miscellaneous
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous


In [ ]:
print(result.head())

           Image           Class
0  image8132.jpg          Jacket
1  image1490.jpg   Miscellaneous
2  image3111.jpg          Jacket
3   image664.jpg  Christmas_Tree
4  image5802.jpg   Miscellaneous


In [ ]:
result.to_csv('out.csv',index = False)

In [ ]:
dff = pd.read_csv('out.csv')

In [ ]:
dff.head()

,Image,Class
0,image8132.jpg,Jacket
1,image1490.jpg,Miscellaneous
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous
